In [24]:
from utils.linter import LingtingReport
from utils.radon_generator import RadonAnalyzer
from utils.results import QualityReport
from utils.bandit_eval import Bandit
from utils.memory_profiler_generator import MemoryProfilerScriptGenerator
import glob
import os

# List of folders to be excluded from analysis
folders_not_to_visit = ["archived", "utils", "app_domain_template", "data"]

algorithms = [x for x in glob.glob("*") if x not in folders_not_to_visit and os.path.isdir(x)]

load_file = lambda file: open(file, "r").read()

In [71]:
from utils.linter import LingtingReport
from utils.radon_generator import RadonAnalyzer
from utils.results import QualityReport
from utils.bandit_eval import Bandit
from utils.memory_profiler_generator import MemoryProfilerScriptGenerator
import glob
import pandas as pd
import os
import sys
import numpy as np
import subprocess


# List of folders to be excluded from analysis
folders_not_to_visit = ["archived", "utils", "app_domain_template", "data"]

# List of algorithms
algorithms = [x for x in glob.glob("*") if x not in folders_not_to_visit and os.path.isdir(x)]

class MuteOutput:
    def __init__(self, mute=True):
        self.mute = mute
        self._stdout = sys.stdout

    def __enter__(self):
        if self.mute:
            sys.stdout = open(os.devnull, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self._stdout

# Loop through each algorithm
for algorithm in algorithms:
    print(f"Analyzing: {algorithm} ({algorithms.index(algorithm)+1}/{len(algorithms)})...\n")
    if algorithm not in ['monte_carlo_simulation']:
        continue

    # Initializing QualityReport() Instance
    report = QualityReport()

    # Initializing Bandit() Instance
    print("Running bandit on all files...")
    # bandit = Bandit(algorithm)

    # Filtering the list of llms to only include valid folders
    algorithm_path = glob.glob(f"{algorithm}/*")
    algorithm_folder = [os.path.isdir(folder) for folder in algorithm_path]
    filtered_list = [value + "/" for value, condition in zip(algorithm_path, algorithm_folder) if condition and value.split('/')[-1] not in folders_not_to_visit]

    # Looping through each LLM
    for llm in filtered_list:
        model = llm.split("/")[-2]
        print(f"Analyzing: {model} ({filtered_list.index(llm)+1}/{len(filtered_list)})...\n")

        # Initializing LingtingReport() Instance
        print("Running linter... (1/3)")
        #linter = LingtingReport(llm)

        # Initializing RadonAnalyzer() Instance
        print("Running radon... (2/3)")
        #radon = RadonAnalyzer(llm)

        # Adding Radon Results to QualityReport
        #report([radon.df, linter.df])

        # Generate memory profiler results
        print("Performing Resource Monitoring... (3/3)")

        # Loop through each script in the resource_monitor folder
        for script in os.listdir(os.path.join(llm, 'resource_monitor')):
            # Skip files that are not python files
            if not script.endswith(".py"):
                continue

            print(f"Executing {script}...")

            # Skip potential files ending with .py
            cpu_usage_list = []
            memory_usage_list = []


            # Loop over the resource monitor 5 times to get the maximum values
            print("Looping")
            for _ in range(5):
                print("Looping: ", _ + 1)
                # Get the path of the script
                script_path = os.path.join(llm, 'resource_monitor', script)

                # Modified line to capture output and errors
                result = subprocess.run(['python3', script_path], check=True, timeout=20, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

                # Decode and process the output
                output = result.stdout.decode('utf-8')
                output_list = output.split('\n')
                output_list = [float(x) for x in output_list if x != '']

                # Append the results to the output list
                cpu_usage_list.append(output_list[0])
                memory_usage_list.append(output_list[1])
            print("Done Looping")
            max_cpu = max(cpu_usage_list)
            max_memory = max(memory_usage_list)

            # Append the results to the output list

            results = [[y, 'psutil', llm.split('/')[-2], x, script.split('_')[0]] for x, y in zip([max_cpu, max_memory], ['cpu','memory_usage'])]
            break
        break
    break
            #resource_frame = pd.DataFrame(result, columns=['metric', 'framework', 'model','value','prompt'])

Analyzing: monte_carlo_simulation (1/9)...

Running bandit on all files...
Analyzing: opencodeinterpreter (1/10)...

Running linter... (1/3)
Running radon... (2/3)
Performing Resource Monitoring... (3/3)
Executing long_resource_version.py...
Looping
Looping:  1
Looping:  2
Looping:  3
Looping:  4
Looping:  5
Done Looping


In [72]:
pd.DataFrame(results, columns=['metric', 'framework', 'model','value','prompt'])
#results

,metric,framework,model,value,prompt
0,cpu,psutil,opencodeinterpreter,26.983333,long
1,memory_usage,psutil,opencodeinterpreter,156.843750,long


In [58]:
import subprocess
import os
import glob

# List of folders to be excluded from analysis
folders_not_to_visit = ["archived", "utils", "app_domain_template", "data"]

# List of algorithms
algorithms = [x for x in glob.glob("*") if x not in folders_not_to_visit and os.path.isdir(x)]

# Output list to store results
output_list = []

# Loop through each algorithm
for algorithm in algorithms:
    if algorithm not in ["pca"]:
        continue

    print(f"Analyzing: {algorithm}")
    # Filtering the list of llms to only include valid folders
    algorithm_path = glob.glob(f"{algorithm}/*")
    algorithm_folder = [os.path.isdir(folder) for folder in algorithm_path]
    filtered_list = [value + "/" for value, condition in zip(algorithm_path, algorithm_folder) if condition and value.split('/')[-1] not in folders_not_to_visit]

    # Looping through each LLM
    for llm in filtered_list:

        # Loop through each script in the resource_monitor folder
        for script in os.listdir(os.path.join(llm, 'resource_monitor')):
            print("Model: ", llm, " Exectueting script: ", script)
            # Skip potential files ending with .py
            if not script.endswith(".py"):
                continue

            cpu_usage_list = []
            memory_usage_list = []

            # Loop over the resource monitor 5 times
            for _ in range(5):
                # Get the path of the script
                script_path = os.path.join(llm, 'resource_monitor', script)

                # Modified line to capture output and errors
                result = subprocess.run(['python3', script_path], check=True, timeout=20, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

                # Decode and process the output
                output = result.stdout.decode('utf-8')
                results = output.split('\n')
                results = [float(x) for x in results if x != '']

                # Append the results to the output list
                cpu_usage_list.append(results[0])
                memory_usage_list.append(results[1])

            max_cpu = max(cpu_usage_list)
            max_memory = max(memory_usage_list)

            # Append the results to the output list
            output_list.append([algorithm, 'cpu', 'psutil', llm.split('/')[-2], max_cpu, script.split('_')[0]])
            output_list.append([algorithm, 'memory_usage', 'psutil', llm.split('/')[-2], max_memory, script.split('_')[0]])

            pd.DataFrame(output_list, columns=['metric', 'framework', 'model','value','prompt'])

Analyzing: pca
Model:  pca/opencodeinterpreter/  Exectueting script:  long_resource_version.py
Model:  pca/opencodeinterpreter/  Exectueting script:  medium_resource_version.py
Model:  pca/opencodeinterpreter/  Exectueting script:  short_resource_version.py
Model:  pca/gpt-35-turbo/  Exectueting script:  long_resource_version.py
Model:  pca/gpt-35-turbo/  Exectueting script:  small_resource_version.py
Model:  pca/gpt-35-turbo/  Exectueting script:  medium_resource_version.py
Model:  pca/gemini/  Exectueting script:  long_resource_version.py
Model:  pca/gemini/  Exectueting script:  medium_resource_version.py
Model:  pca/gemini/  Exectueting script:  short_resource_version.py
Model:  pca/codellama/  Exectueting script:  long_resource_version.py
Model:  pca/codellama/  Exectueting script:  small_resource_version.py
Model:  pca/codellama/  Exectueting script:  medium_resource_version.py
Model:  pca/wizardcoder/  Exectueting script:  long_resource_version.py
Model:  pca/wizardcoder/  Exect

In [59]:
#out =
nut = pd.DataFrame(output_list, columns=['metric', 'framework', 'model','value','prompt'])

In [60]:
nut

,algorithm,metric,framework,model,value,prompt
0,pca,cpu,psutil,opencodeinterpreter,86.458333,long
1,pca,memory_usage,psutil,opencodeinterpreter,188.921875,long
2,pca,cpu,psutil,opencodeinterpreter,78.766667,medium
3,pca,memory_usage,psutil,opencodeinterpreter,323.437500,medium
4,pca,cpu,psutil,opencodeinterpreter,86.966667,short
5,pca,memory_usage,psutil,opencodeinterpreter,311.781250,short
6,pca,cpu,psutil,gpt-35-turbo,88.833333,long
7,pca,memory_usage,psutil,gpt-35-turbo,340.875000,long
8,pca,cpu,psutil,gpt-35-turbo,89.108333,small
9,pca,memory_usage,psutil,gpt-35-turbo,340.796875,small


In [49]:
max([1,2,4,5])

5

In [48]:
out.sort_values(by='value', ascending=False)

,algorithm,metric,framework,model,value,prompt
41,monte_carlo_simulation,memory_usage,psutil,deepseeker-coder,1239.500000,medium
27,monte_carlo_simulation,memory_usage,psutil,wizardcoder,305.750000,medium
397,convolution,memory_usage,psutil,opencodeinterpreter,235.390625,short
1,monte_carlo_simulation,memory_usage,psutil,opencodeinterpreter,156.937500,long
445,convolution,memory_usage,psutil,magicoder,151.203125,medium
...,...,...,...,...,...,...
282,huffman,cpu,psutil,opencodeinterpreter,0.183333,medium
92,pca,cpu,psutil,baseline,0.000000,baseline
4,monte_carlo_simulation,cpu,psutil,opencodeinterpreter,0.000000,short
54,monte_carlo_simulation,cpu,psutil,magicoder,0.000000,short


In [16]:
import pandas as pd

df = pd.DataFrame(output_list, columns=['model', 'prompt', 'cpu', 'memory'])

In [17]:
df['llm'] = df.llm.apply(lambda x: x.split('/')[-2])
df['prompt'] = df.prompt.apply(lambda x: x)

In [31]:
radon.df

,metric,framework,model,value,prompt
0,loc,pylint,magicoder,34.000000,medium
0,sloc,pylint,magicoder,21.000000,medium
0,comments,pylint,magicoder,9.000000,medium
0,multi,pylint,magicoder,0.000000,medium
0,blank,pylint,magicoder,11.000000,medium
0,comments_to_loc,pylint,magicoder,0.264706,medium
0,comments_to_sloc,pylint,magicoder,0.428571,medium
0,multi_and_comments_to_loc,pylint,magicoder,0.264706,medium
0,max_complexity,pylint,magicoder,1.000000,medium
0,number_of_methods,pylint,magicoder,3.000000,medium


In [ ]:
['cpu', 'psutil',model, 'value', prompt]

In [30]:
output_list

[['monte_carlo_simulation/opencodeinterpreter/',
  'long_resource_version.py',
  17.141666666666666,
  154.46875],
 ['monte_carlo_simulation/opencodeinterpreter/',
  'medium_resource_version.py',
  10.816666666666668,
  37.96875],
 ['monte_carlo_simulation/opencodeinterpreter/',
  'short_resource_version.py',
  0.0,
  77.171875],
 ['monte_carlo_simulation/gpt-35-turbo/',
  'long_resource_version.py',
  14.450000000000001,
  56.234375],
 ['monte_carlo_simulation/gpt-35-turbo/',
  'medium_resource_version.py',
  24.108333333333334,
  50.953125],
 ['monte_carlo_simulation/gpt-35-turbo/',
  'short_resource_version.py',
  15.966666666666667,
  35.625],
 ['monte_carlo_simulation/gemini/',
  'long_resource_version.py',
  15.333333333333334,
  76.0],
 ['monte_carlo_simulation/gemini/',
  'medium_resource_version.py',
  15.458333333333334,
  113.984375],
 ['monte_carlo_simulation/gemini/',
  'short_resource_version.py',
  14.633333333333333,
  36.5],
 ['monte_carlo_simulation/codellama/',
  'lo

In [ ]:
aresult = subprocess.run(['python3', script_path], check=True, timeout=20, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [42]:
result

CompletedProcess(args=['python3', 'quick_sort/deepseeker-coder/resource_monitor/medium_resource_version.py'], returncode=0, stdout=b'[1, 1, 2, 3, 6, 8, 10]\n19.8\n36.359375\n', stderr=b'')